In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Lab 01 CUDA-Q Quick Start**

This lab introduces the basic syntax of CUDA-Q. You will learn how to define qubits, construct qauntum circuits, and measure the outputs of those circuits using three different methods.

These concepts are all you need to begin coding!

**What you'll do:**

1.   Define and visualize single-qubit and mult-qubit states and operations
2.   Write your first quantum programs
3.   Interpret the results of a quantum program

$
\renewcommand{\ket}[1]{|{#1}\rangle}
\renewcommand{\bra}[1]{\langle{#1}|}
$

In [ ]:
## Instructions for Google Colab.
# Uncomment the lines below and execute the cell to install cuda-q

# !pip install cudaq
# !pip install qutip\>=5 matplotlib\>=3.5

#!wget -q https://github.com/nvidia/cuda-q-academic/archive/refs/heads/main.zip
#!unzip -q main.zip
#!mv cuda-q-academic-main/quick-start-to-quantum/interactive_widget ./interactive_widget

In [ ]:
# Necessary packages

import cudaq
import qutip
import numpy as np
import matplotlib.pyplot as plt




---


**Background**

We've already introduced the states $\ket{0}$ and $\ket{1}$.  We can identify $\ket{0}$ with the vector $\begin{pmatrix} 1 \\ 0\end{pmatrix}$ and $\ket{1}$ with the vector $\begin{pmatrix} 0\\ 1\end{pmatrix}$.  Then, for the purposes of this tutorial, any other quantum state $\ket{\psi}$ on the Bloch sphere can be written as a linear combination of $\ket{0}$ and $\ket{1}$ .  In other words, quantum states take the form

$$
\ket{\psi} = \alpha\ket{0}+\beta\ket{1},
$$

where $\alpha$ and $\beta$ are complex numbers satisfying the equation $|\alpha|^2+|\beta|^2 = 1$. The coefficients $\alpha$ and $\beta$ are referred to as *probability amplitudes*, or *amplitudes* for short. The restriction that $|\alpha|^2+|\beta|^2 = 1$ ensures that the state is on the Bloch sphere.

Let's see how the expression $
\ket{\psi} = \alpha\ket{0}+\beta\ket{1},
$ maps to the Bloch sphere. First, we rewrite the coefficients $\alpha$ and $\beta$ in the following manner:

$$
\alpha = \cos(\frac{\theta}{2})\text{ and }\beta = \sin(\frac{\theta}{2})e^{i\varphi},
$$
where $\theta$ is a value in the interval $[0,\pi]$ and $\varphi$ is a value in the interval $[0,2\pi)$. Using spherical coordinates we can draw the state on the Bloch sphere following the convention in the image below:

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/main/images/BlochsphereAngles.png" alt="Image of a State on the Bloch Sphere with spherical coordinates indicated" width="200"/>

Use the interactive widget below to visualize $\ket{\psi} = \cos{\frac{\theta}{2}}\ket{0}+\sin{\frac{\theta}{2}}e^{i\varphi}\ket{1}$ for various values of $\theta$ and $\varphi$.  As you experiment, think about the following questions: What effect does changing $\varphi$ have on the position of the vector on the Bloch Sphere?  What combination of values of $\theta$ and $\varphi$ produces a state pointing along the $x$-axis?

<iframe src="https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/quantum_coin_widget.html" width="800" height="600"></iframe>

> **Note:** If the widget does not appear above, you can access it directly using [this link](https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/quantum_coin_widget.html).
---

## 1.0 Using CUDA-Q to define and visualize a quantum state



We are now ready to use CUDA-Q!

> **FAQ:** *What is CUDA-Q?*

> **Answer:** [CUDA-Q](https://developer.nvidia.com/cuda-q) is a platform designed for hybrid application development. That is, CUDA-Q allows programming in a heterogeneous environment that leverages not only quantum processors and quantum emulators, but also CPUs and GPUs. CUDA-Q is interoperable with CUDA and the CUDA software ecosystem for GPU-accelerated applications. CUDA-Q consists of both C++ and Python extensions. In these notebooks, we'll use Python.

 Let's use CUDA-Q to create the quantum states of single qubits and visualize them on the Bloch sphere.  CUDA-Q uses the `cudaq.kernel` decorator on functions to define quantum states, and, as we will see later, CUDA-Q kernels can also define quantum circuits.

 ### 1.1 Visualizing the minus state

The state

 $$
 |-\rangle = \frac{1}{\sqrt{2}}|0\rangle-\frac{1}{\sqrt{2}}|{1}\rangle$$ can be rewritten as

The state $|-\rangle$ is a vector pointing in the direction of the negative $x$-axis, which gives some explanation for the jargon: "minus state."

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/main/images/BlochsphereMinus.png" alt="Minus state represented on the Bloch Sphere" width="200"/>

In [ ]:
# Defining the minus state in CUDA-Q

# First we define a vector of complex numbers
# for the coefficients alpha and beta
# of the state |psi> = alpha|0> + beta|1>,
# where alpha = 1/sqrt(2)+0j and beta = -1/sqrt(2)+0j

c = [complex(np.sqrt(2)/2, 0), complex(-np.sqrt(2)/2,0)]

# Define a cudaq.kernel to represent the minus state
@cudaq.kernel
def minus_state():
    q = cudaq.qvector(c)

Once we have defined a kernel, we can call upon the get_state command to read out the state from the kernel. Then, we can add this state to a Bloch sphere using add_to_bloch_sphere which is displayed with the show command and the file can be saved using the save option.

In [ ]:
# Visualizing a state in CUDA-Q

# Define a sphere object representing the state of the single qubit
sphere = cudaq.add_to_bloch_sphere(cudaq.get_state(minus_state))

# Display the Bloch sphere
cudaq.show(sphere)


# 1.2 Writing your first quantum programs

Now that we understand qubits and have created quantum state representations in CUDA-Q, let's move on to actual quantum computation. We're ready to write our first quantum program!

The general structure of a quantum program is:
* Encode information into the quantum state by initializing qubit(s)
* Manipulate the quantum state of the qubit(s) with quantum gate(s)
* Extract information from the quantum state by measuring the state of the qubit(s)

These three steps are outlined in the diagram below:

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/main/images/circuit.png" alt="image of a quantum circuit with the three partsL encode information, manipulate quantum states, extract information" width="200"/>

In this section, we'll look at a few simple examples of quantum programs to illustrate these steps.

## 1.2.1 Bit Flip

Let's walk through the three steps of writing the Bit Flip program in CUDA-Q. This program is a quantum version of the classical `NOT` operation that flips a bit from $0$ to $1$ and vice versa.

Outline of the Bit Flip program: Going from $|0\rangle$ to $|1\rangle$.
* Initialize the zero state
* Manipulate the quantum state by applying the bit flip gate (the `x` gate)
* Measure the qubit

Let's start by defining a kernel and specifying that this kernel only contains one qubit.  

> **CUDA-Q Quick Tip:**  The `cudaq.qvector` command has two purposes.  When `cudaq.qvector` accepts a `list` as we saw in the `minus_state` example in the previous section, the kernel is initialized in the state corresponding to the coefficients from the list.  In the example below, `cudaq.qvector` will accept an `int` value and will allocate that number of qubits to the kernel.  These qubits, by default, are initialized in the zero-state.

In [ ]:
@cudaq.kernel
def bitflip():
    # Allocate 1 qubit initialized in the zero state
    qubit = cudaq.qvector(1)

   ###TODO(FINISH)

# Visualizing a state of the bitflip kernel before the x gate is applied

# Define a sphere object representing the state of the single qubit
sphere = cudaq.add_to_bloch_sphere(cudaq.get_state(bitflip))

# Display the Bloch sphere
cudaq.show(sphere)


In general, we can change a quantum state through multiplication by a unitary matrix. (The definition of a unitary matrix is not critical for this notebook, but if you're curious you can learn more in [chapter 1 of Quantum Computer Science](http://mermin.lassp.cornell.edu/qcomp/CS483.html).)

In the context of a quantum program, we refer to matrix multiplication by a unitary matrix $U$ as applying a $U$-gate.   In CUDA-Q, the $X$-gate is implemented with the syntax `x`.

> **CUDA-Q Quick Tip:** You can view all the built-in quantum gate operations in CUDA-Q [here](https://nvidia.github.io/cuda-quantum/latest/api/default_ops.html#unitary-operations-on-qubits).


In [ ]:
@cudaq.kernel
def bitflip():
    # Allocate 1 qubit initialized in the zero state
    qubit = cudaq.qvector(1)

    # apply the x-gate (bit flip gate)
    x(qubit[0])


sphere = cudaq.add_to_bloch_sphere(cudaq.get_state(bitflip))
cudaq.show(sphere)


## 1.2.1 visualizing quantum circuits
The process of visualizing the state of a kernel on a Bloch sphere using `get_state` is not very efficient and won't be useful when we have many qubits.

Firstly, Bloch spheres can only represent the state of a single qubit.  But also, the `get_state` command may be overkill for the amount of information we can or need to recover from several qubits.  Remember, to describe the state of $n$ qubits we need to compute $2^n$ probability amplitudes. This may be prohibilitively expensive to compute in terms of time and memory resources.

There are other visualization tools that we can use to check the kernels that we create.  Some kernels can be represented as
quantum circuit diagrams.

CUDA-Q includes the `cudaq.draw` command to generate an ascii image of the circuit diagram of a kernel.  Each row of the circuit diagram represents a qubit. In this case, we only have one qubit, which by default is named `q0` in the diagram. Operations applied to the qubit are shown as boxes and are read from left to right. Our kernel has only one gate, the `x` gate.

In [ ]:
print(cudaq.draw(bitflip))

> **FAQ:** *What's the difference between a quantum kernel and a quantum circuit?*

> **Answer:** Quantum kernels are more general than quantum circuits.  That is, every quantum circuit is a quantum kernel, but not every quantum kernel is a quantum circuit. Quantum kernels are defined as functions that are executed on a quantum processing unit (QPU) or a simulated QPU. Quantum kernels can be combined with classical functions to create quantum-classical applications that can be executed on a heterogeneous system of QPUs, GPUs, and CPUs to solve real-world problems.

## 1.2.2 Sampling our bit flip circuit
The final step of our first quantum program is to extract information from the quantum state of the qubit.  This is done with the `sample` command. The sample command runs the kernel and takes a measurement `shots_count`-many times

In our example, applying the `x`-gate to the zero state will always result in the one state, unless of course there are errors in the circuit execution.  For now, we'll assume that all of our kernel executions are error free.  In this case, we would expect to see the one state returned for each kernel execution. Run the cell block below to see the results.

In [ ]:
# Sampling the bit flip kernel
shots = 123
results = cudaq.sample(bitflip, shots_count=shots)

# The output of the sample command looks like a dictionary, but is of a different type
print("Results from sampling {} times: {}\n".format(shots, results))
print("Although the results appear to be of type dict, \n the type of the results from the sample \n command is {}.\n".format(type(results)))

# Often it will be useful to identify the most probable outcome
# and the probability of this outcome
most_probable_result = results.most_probable()
probability = results.probability(most_probable_result)
print("Most probable result: " + most_probable_result)
print("Measured with probability " + str(probability), end='\n\n')

## 1.3 Hello World example

To get a better appreciation how quantum programs differ from classical programs, let's look at a Hello World program, which revisits the minus state kernel.

Hello World: Generate and measure the Minus State
* Initialize one qubit in the one-state
* Manipulate the quantum state by transforming it into the minus state
* Extract information from the quantum state by taking measurement(s)

The Hadamard operator is the matrix $$H =  \frac{1}{\sqrt{2}}\begin{pmatrix} 1&  1 \\ 1 &  -1\end{pmatrix}.$$

Notice that if we multiply $H$ and $|{1}\rangle$ we get $|{-}\rangle$:  

$$ H|{1}\rangle =  \frac{1}{\sqrt{2}}\begin{pmatrix} 1&  1 \\ 1 &  -1\end{pmatrix}\begin{pmatrix} 0 \\ 1 \end{pmatrix} = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\   -1\end{pmatrix} = \frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle=|{-}\rangle.$$

This calculation suggests a way to generate the minus state:

* First intialize the one state
* Apply the Hadamard gate using the CUDA-Q command `h`.  

We now have a plan, so let's code it up.

In [ ]:
# Kernel for the minus state

@cudaq.kernel
def minus_kernel():
    # Allocate a qubit
    qvector = cudaq.qvector(1)

    # Initialize the state |1>
    x(qvector)

    # Apply the Hadamard gate
    h(qvector)

# Draw the circuit to check our work
print(cudaq.draw(minus_kernel))

In [ ]:
# Sampling the minus kernel
shots = 123
results = cudaq.sample(minus_kernel, shots_count=shots)
print("Results from sampling {} times: {}".format(shots, results))

# # Often it will be useful to identify the most probable outcome
# # and the probability of this outcome
# most_probable_result = results.most_probable()
# probability = results.probability(most_probable_result)
# print("Most probable result: " + most_probable_result)
# print("Measured with probability " + str(probability), end='\n\n')

## 1.4 Interpreting the results of a quantum program

In [ ]:

# Kernel for the plus state
@cudaq.kernel
def plus_kernel():

    # Allocate a qubit
    # Initialize the state |0>
    qubits = cudaq.qvector(1)
    # Apply the Hadamard gate
    h(qubits[0])

# Draw the circuit to check our work
print(cudaq.draw(plus_kernel))

# Sampling the plus kernel
shots = 1024
results = cudaq.sample(plus_kernel, shots_count=shots)
print(results)



How do the results of sampling the `plus_kernel` compare to those obtained for the `minus_kernel` in the previous section?  What does this seem to apply concerning our ability to distinguish $|{+}\rangle$ from $|{-}\rangle$?  

If qubits store information, we would like to be able distinguish two qubits that might be storing different information. For example we might want to distinguish the one-qubit state $|{+}\rangle$  from the state $|{-}\rangle$ . We'll resolve this issue in the next example by changing the way we measure the two qubits.


Exercise 4: Create two quantum kernels to sample. The first will prepare the  |+⟩  state and then measure with mx. The second kernel will prepare the  |−⟩  state and then measure with mx.

In [ ]:
# EXERCISE 4

shots = 10

@cudaq.kernel
def plus_measure_x() -> bool:
    q =cudaq.qvector(1)
    ## EDIT CODE BELOW THIS LINE

    ## EDIT CODE ABOVE THIS LINE
    # Specify measurement in either the x,y or z basis with mx, my, or mz
    measurement = mx(q[0])
    return measurement

@cudaq.kernel
def minus_measure_x() -> bool:
    q =cudaq.qvector(1)
    ## EDIT CODE BELOW THIS LINE

    ## EDIT CODE ABOVE THIS LINE
    measurement = mx(q[0])
    return measurement

# Use the run command when measurements are specified in the kernel
results_plus_measure_x = cudaq.run(plus_measure_x, shots_count=shots)
print("Results from running the plus_hadamard kernel {} times: {}".format(shots, results_plus_measure_x))

results_minus_measure_x = cudaq.run(minus_measure_x, shots_count=shots)
print("Results from running the minus_hadamard kernel {} times: {}".format(shots, results_minus_measure_x))

>**CUDA-Q Quick Tip:** By default, when using the `cudaq.sample` execution mode, qubit measurements are conducted on the standard computational basis, which is along the $z$-axis.  In the cell block below, we have added the measurement explicitly with the `mz` command. There are other measurement options `mx` and `my` built into CUDA-Q that we'll experiment with in the next section. When we specify the measurement, instead of using the `cudaq.sample` execution mode, we switch to the `cudaq.run` execution mode.  To learn more about the differences between these execution modes, see the [documentation](https://nvidia.github.io/cuda-quantum/latest/using/examples/executing_kernels.html).

Notice that the results from running the two kernels above differ from one another.  And thus, by switching to measurement with the `mx` command we are able to distinguish the $|+\rangle$ and $|-\rangle$ states. However, there is a drawback &#8212; measuring with `mx` doesn't allow us to distinguish $|0\rangle$ from $|1\rangle$, because of the Uncertainty Principle in quantum mechanics. Test this claim out for yourself by creating and running kernels for the $|0\rangle$ and $|1\rangle$ states using the `mx` measurements.

What is happening when we take a measurement?  What distinguishes the `mz` command from the `mx` command?  When we  measure a state $|{\psi}\rangle=\alpha|0\rangle+\beta|1\rangle$ with `mz`we collapse the state to $|{0}\rangle$, with a probability $|\alpha|^2$  and will collapse the state of the qubit to  $|1\rangle$ with probability $|\beta|^2$. You can graphically visualize this collapse under a `mz` measurement on the Bloch sphere as projecting the state $|{\psi}\rangle$ onto the $z$ axis. Analogously, when we measure with `mx` we collapse our state $|\psi\rangle$ to either $|+\rangle$ or $|-\rangle$ along the $x$-axis of the Bloch sphere.


# 2.0 Program with more than one qubit

## 2.1 Notation for a 2-qubit state
$\renewcommand{\ket}[1]{|#1\rangle}$
In Lab 1, we saw that the quantum state of a single qubit could be written as linear combinations of the states $|0\rangle$ and $|1\rangle$.  The states $|0\rangle$ and $|1\rangle$ are often referred to as the computational basis states of a single qubit.  When we have 2 qubits, we will need $2^2 = 4$ basis states to describe all possible quantum states of 2 qubits because each qubit could be measured as either a $0$ or a $1$.  The computational basis states used to describe a state of 2 qubits are
$$|{00}\rangle, |{01}\rangle, |{10}\rangle,  \text{ and }|{11}\rangle.$$

##2.2 All in one
In this section, we'll write a program that builds upon the Bit-Flip and Hello World (minus state) programs. Our goal is to create one program with 2 qubits.  The first qubit will be initialized in the zero state, and we'll flip it to the one state.  The second qubit will be initialized in the minus state, and just for fun we'll apply the bit flip gate `x` to it too. The diagram represents the circuit for this program.

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/refs/heads/main/images/2-qubit-circuit.png" alt="quantum circuit diagram with 2 qubits" width="200"/>

The statevector of the 2-qubit system after the gates are applied, but  prior to measurement should be $$|{\psi}\rangle = -\frac{1}{\sqrt{2}}|{10}\rangle +\frac{1}{\sqrt{2}}|{11}\rangle = \begin{pmatrix}0 \\ 0 \\ -\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix}$$

In [ ]:
import cudaq
import numpy as np

In [ ]:
# Writing the all-in-one kernel

@cudaq.kernel
def all_in_one():
    """Creates a kernel with 2 qubits. The first qubit is initialized
    in the zero state, the second qubit is initialized in the minus state.
    An x gate is then applied to each qubit.
    """
    # Allocate 2 qubits (indexed by 0 and 1)
    qvector = cudaq.qvector(2) # By default these qubits are the zero state

    # Initialize qubit indexed with 1 as the minus state
    x(qvector[1])
    h(qvector[1])

    # Manipulate the quantum states by applying the bit flip gate (the `x` gate) to each qubit
    x(qvector) # We can apply a single-qubit gate to each qubit in a list of qubits

print(cudaq.draw(all_in_one))


Let's check that this kernel generates our target state $\ket{\psi} = -\frac{1}{\sqrt{2}}\ket{10} +\frac{1}{\sqrt{2}}\ket{11}$ by using the `get_state` command.  We'll also use the `get_state.amplitudes` option to extract out the coefficients of the computational basis states $\ket{10}$ and $\ket{11}$ that are of interest to us.

In [ ]:
# Compute the state of the system prior to measurement
all_in_one_state = cudaq.get_state(all_in_one)

# Return the amplitudes of |10> and |11>
all_in_one_amplitudes = all_in_one_state.amplitudes([[1,0], [1,1]])

# Print
precision = 4
print('All-in-one statevector array of coefficients:', np.round(np.array(all_in_one_state), precision))
print('All-in-one statevector: {} |10> + {} |11>'.format(np.round(all_in_one_amplitudes[0],precision), np.round(all_in_one_amplitudes[1],precision)))


## 2.3 Kernels as subroutines

In this section, we will create the same quantum circuit as above, but this time we'll use kernels as subroutines. We'll create three separate kernels:

* `minus`, a kernel for the subroutine that initializes a given qubit as the minus state
* `xgate`, a kernel that applies the `x` gate to a list of qubits
* `nested_quantum_program`, a kernel to allocate the qubits and call the `minus` and `xgate` subroutines

Let's see how this is done. Take note of how we can pass qubits and lists of qubits to the kernel functions.

In [ ]:
# Defining three kernels to generate the example circuit

# Defining the subroutine, minus, as a kernel
@cudaq.kernel
def minus(qubit: cudaq.qubit):
    """When applied to a qubit in the zero state, generates the minus state
    Parameters
    ----------
    qubit : cudaq.qubit
        qubit upon which the kernel instructions will be applied
    """
    ### EDIT CODE BELOW HERE###
    x(qubit)
    h(qubit)
    ### EDIT CODE ABOVE HERE###


# Defining the subroutine, xgate, as a kernel
@cudaq.kernel
def xgate(qubits: cudaq.qvector):
    """Applies an x-gate to each qubit in qubits
    Parameters
    ----------
    qubits : cudaq.qvector
        list of qubits upon which the kernel instructions will be applied
    """
    x(qubits)

# Allocating qubits and calling the subroutines defined above
@cudaq.kernel
def nested_quantum_program(num_qubits: int):
    """Creates a kernel with num_qubits-many qubits. The first qubit is initialized
    in the zero state, the second qubit is initialized in the minus state.
    An x gate is then applied to each qubit. Here num_qubits is at least 2.
    Parameters
    ----------
    num_qubits : int
        Number of qubits to be allocated
    """

    # Allocate num_qubits qubits indexed from 0 to num_qubits-1
    qvector = cudaq.qvector(num_qubits) # By default these qubits are in the zero state

    # Initialize the qubit indexed with 1 as the minus state
    # by calling the minus kernel acting on qvector[1]
    minus(qvector[1])

    # Manipulate the quantum states by applying the bit flip gate (the `x` gate) to each qubit
    # by calling the xgate kernel applied to a list of qubits
    xgate(qvector)


num_qubits = 2 # num_qubits should be an integer > 1

print(cudaq.draw(nested_quantum_program,num_qubits))

### 2.4 Write a "Hello, Entangled World!" program

So far, all of the quantum operations (e.g. `x`, `h`) that we programmed acted only on an individual qubit.  In this section, we'll write a program using multi-qubit gates.  Let's start with the example of the controlled-not gate, often abbreviated the `CNOT` gate. This is a gate that operates on 2 or more qubits. One of the qubits is considered the "target", and the remaining qubits are the "control".  For instance `CNOT([q0,q1],q2)` would represent a gate with control qubits in the list `[q0,q1]` and target qubit `q2`.  

In a circuit diagram we depict the `CNOT` gate as a line connecting the control and target qubits with a $\bigoplus$ located at the target qubit and a solid circle on control qubits. For instance, in the circuit diagram below, we see a `CNOT(q0, q1)` and a `CNOT([q1,q2],q0)` gate.

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/refs/heads/main/images/cnots.png" alt="quantum circuit diagram with various cnot gates" width="200"/>

The rough idea is that if the "control" qubits are all in the state  |1⟩  then an application of the CNOT gate will apply a bitflip (i.e. an x gate) to the target qubit. If on the other hand, at least one of the "control" qubits is in the  |0⟩  state the CNOT gate has no effect.

To illustrate this, let's code up a few examples.

Quick Tip: The syntax for the CNOT gate is x.ctrl(control_qubit, target_qubit) if we have only one control qubit. When we have multiple control qubits, we send them as a list to the x.ctrl function: x.ctrl(control_qubits_list, target_qubit).


**Exercise 4 Part a:** Edit the code block below to see the effect of the gate sequence in the diagram below on the qubits if $q_0$ was initialized as $\ket{+}$ and $q_1$ was initialized as $\ket{1}$

<img src="https://raw.githubusercontent.com/NVIDIA/cuda-q-academic/main/images/exercise3a.png" alt="image of a circuit with 3 alternating cnots and q_0 initialized as |+> and q_1 initialized as |1>" width="200"/>

In [ ]:
# EXERCISE 3 Part a
@cudaq.kernel
def init(qubits: cudaq.qvector):
    """Kernel for initializing the qubits. Notice how cudaq.qvector objects are passed"""

    # Initialize qubits q0 and q1 in the plus and one states, respectively

    # Edit code below this line

    # Edit code above this line

@cudaq.kernel
def alternating_cnots(qubit0: cudaq.qubit, qubit1: cudaq.qubit):
    """Apply a sequence of 3 CNOTs with alternating controls and targets on the 2 qubits given as arguments.
       Notice how individual qubit objects are passed. """
    # We will see later that it doesn't matter which qubit you start out with as the first control, as long as
    # you alternate the control and target qubits with each of the 3 applications of the cnot gate

    # Edit code below this line

    # Edit code above this line


@cudaq.kernel
def three_alternating_cnots():
    """Kernel for the circuit drawn above"""

    # Allocate qubits
    q = cudaq.qvector(2)

    # Initialize qubits q0 and q1 in the plus and one states, respectively
    init(q)

    # # Apply alternating CNOTs
    alternating_cnots(q[0], q[1])



results = cudaq.sample(three_alternating_cnots)
print('The distribution of states after sampling is: {}'.format(results))